In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import datetime
import gmaps
import os

# # Incorporated citipy to determine city based on latitude and longitude
# from citipy import citipy

# # Import API key
# from api_keys import g_key
# gmaps.configure(api_key=g_key)

In [2]:
# Make a reference to the starter csv file path and create df
pprep_df = pd.read_csv('../Resources/Texas_Prison_Data.csv') 
pprep_df.head()


,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,068770,Grayson,35990003,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS
1,524524,758486,"HARPER,DAVID JOHN JR",Estelle,M,W,91,01/01/9999,01/01/9999,08/03/2031,95-765-C,McLennan,36010001,INDECENCY WITH A CHILD,08/13/1996,09/15/1995,Life,None,08/03/2031,NOT IN REVIEW PROCESS
2,703267,841623,"RODRIGUEZ,THEODORE",Beto,M,W,82,08/03/2043,08/03/2043,02/01/2021,97-396-C,McLennan,11990003,AGG SEX ASLT,08/05/1998,06/01/1996,45.0,None,02/01/2021,IN PAROLE REVIEW PROCESS
3,708520,256174,"ALONZO,BENITO",Hospital Galveston,M,H,85,01/01/9999,01/01/9999,11/25/1985,49995,Travis,49030000,INTRODUCE DEADLY WPN PENAL INSTI,02/27/1976,06/01/1974,Life,Denied on 12/20/2019,12/2020,IN PAROLE REVIEW PROCESS
4,766785,365547,"BROWN,NARRIES EARL",Jester IV,M,W,84,01/01/9999,01/01/9999,08/24/2003,F83-89728-HI,Dallas,11220000,AGG SEX ABUSE CHILD/U14,08/24/1983,03/15/1983,Life,Denied on 12/05/2016,Unavailable at this time.,NOT IN REVIEW PROCESS


In [3]:
offense_df = pd.read_csv('../Resources/updatedoffenseCodes_csv.csv', encoding="utf-8")
offense_df

,Code,Offense_Category,Suggested Bin,Offense,Citation,Statute,L/D
0,3990001,100,bin_01_human_trafficking,MISREPRESENT CHILD AS FAMILY MEMBER ENTRY PORT,37.082(c),PC,MB
1,9990017,200,bin_02_murder,MANSLAUGHTER,19.04,PC,F2
2,9990018,200,bin_02_murder,CRIMINAL NEGLIGENT HOMICIDE,19.05,PC,FS
3,9990020,200,bin_02_murder,CAPITAL MURDER OF A PEACE OFFICER OR FIREMAN,19.03(a)(1),PC,FX
4,9990021,200,bin_02_murder,MURDER UNDER INFLUENCE OF SUDDEN PASSION,19.02(d),PC,F2
...,...,...,...,...,...,...,...
3100,73991005,6200,NaN,NaN,NaN,NaN,NaN
3101,73991032,6200,NaN,NaN,NaN,NaN,NaN
3102,73991058,6200,NaN,NaN,NaN,NaN,NaN
3103,73991073,6200,NaN,NaN,NaN,NaN,NaN


In [4]:
# Create project specific df for merging
offproject_df = offense_df.loc[:,["Code", "Offense_Category"]]
offproject_df

,Code,Offense_Category
0,3990001,100
1,9990017,200
2,9990018,200
3,9990020,200
4,9990021,200
...,...,...
3100,73991005,6200
3101,73991032,6200
3102,73991058,6200
3103,73991073,6200


In [5]:
newoffocde_df = offproject_df.rename(columns={"Code": "Offense Code"})

In [6]:
newoffocde_df

,Offense Code,Offense_Category
0,3990001,100
1,9990017,200
2,9990018,200
3,9990020,200
4,9990021,200
...,...,...
3100,73991005,6200
3101,73991032,6200
3102,73991058,6200
3103,73991073,6200


In [7]:
binlookup_df = pd.merge_ordered(pprep_df, newoffocde_df, fill_method='ffill', left_by='Offense Code')
binlookup_df

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,...,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status,Offense_Category
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,...,Grayson,35990003,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200
1,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,...,Grayson,35990003,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200
2,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,05/02/2021,06/18/2033,03/08/2016,...,Dallas,35990003,UNLAW POSS WIT C/S-METHAM,04/06/2006,09/03/2004,25.0,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200
3,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,05/02/2021,06/18/2033,03/08/2016,...,Dallas,35990003,UNLAW POSS WIT C/S-METHAM,04/06/2006,09/03/2004,25.0,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200
4,1409034,729430,"LERMA,GILBERT LOPEZ",Wynne,M,H,70,01/01/9999,01/01/9999,08/22/2020,...,Williamson,35990003,DEL C/S,09/19/1995,09/28/1994,Life,Denied on 07/16/2020,07/2021,NOT IN REVIEW PROCESS,2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214117,50709066,2144543,"WERKMEISTER,DEBORAH JANE",Hilltop,F,W,55,11/01/2020,06/22/2027,07/23/2018,...,Ellis,23990093,THEFT BY PUBLIC SERVANT,06/23/2017,05/01/2009,10.0,Approved on 06/12/2020,NaN,NOT IN REVIEW PROCESS,1600
214118,50776498,2298374,"NINO-RUIZ,JOSE EDUARDO",Lychner,M,H,29,12/05/2021,12/05/2021,NaN,...,Starr,36060001,BESTIALITY,12/09/2019,03/15/2019,2.0,NaN,NaN,NaN,400
214119,50776498,2298374,"NINO-RUIZ,JOSE EDUARDO",Lychner,M,H,29,12/05/2021,12/05/2021,NaN,...,Starr,36060001,BESTIALITY,12/09/2019,03/15/2019,2.0,NaN,NaN,NaN,400
214120,50797586,2069435,"TORRES,STEPHEN DOUGLAS",Lynaugh,M,H,51,12/01/2020,11/23/2023,NaN,...,El Paso,38040003,BIGAMY,05/05/2016,12/30/2014,8.0,Denied on 12/16/2019,12/2020,IN PAROLE REVIEW PROCESS,2500


In [ ]:
binlookup_df.isna().sum()